In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

img = cv2.imread('img/paper.jpg')
rows, cols = img.shape[:2]

#gray scale
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#가우시안블러로 노이즈 제거
# (5,5): 블러 시킬 커널의 크기
# 0: 자동으로 표준편차 값을 선택해줌
dst = cv2.GaussianBlur(imgray, (5,5),0)

#케니 엣지로 경계 검출
#100, 200 = threshold1, threshold2 : 스레시홀딩에 사용할 최소, 최댓값
edges = cv2.Canny(dst, 100, 200)

#경계 검출한 이미지 'edges'에서 컨투어 찾기
_, contours,_ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
# print(len(contours))
contour = contours[0] #print했을 때 contour가 1개이므로 0번째를 contour로 설정
# mnt = cv2.moments(contour) #moment는 24개의 값을 가짐 - 공간, 중심, 평준화된 중심 모멘트
# print(mnt)
#drawcontour하면 이미지에서 테두리에 초록색 선 확인 가능
#3은 선 두께
draw_img = img.copy()
cv2.drawContours(draw_img, contour, -1, (0,255,0), 3)

#컨투어를 apporxPolyDP() 함수로 단순화 한 뒤 꼭지점 4개 구하기
#종이 안에 글자가 많더라도 approxPolyDP()를 사용하면 글자를 컨투어로 인식하지 않고 
#꼭지점을 줄여 테두리만 컨투어로 인식
epsilon = 0.01 * cv2.arcLength(contour, True)
approx = cv2.approxPolyDP(contour, epsilon, True)
app_img = img.copy()
cv2.drawContours(app_img, [approx], -1, (0,0,255), 3)
print(approx)

#approx 값을 4x2 배열로 reshape
approx = approx.reshape(4,2)

#img에 꼭지점을 원으로 찍어준 것 - 상하좌우 찾아주기 위해
cir_img = img.copy()
cv2.circle(cir_img,tuple(approx[0]),10, (255,0,0), -1) #Left_Top
cv2.circle(cir_img,tuple(approx[1]),10, (0,255,0), -1) #Left_Low
cv2.circle(cir_img,tuple(approx[2]),10, (0,0,255), -1) #Right_Low
cv2.circle(cir_img,tuple(approx[3]),10, (0,255,255), -1) #Right_Top

lt = approx[0]
ll = approx[1]
rl = approx[2]
rt = approx[3]

pts1 = np.float32([lt,rt,ll,rl])
pts2 = np.float32([[0,0], [cols,0], [0, rows], [cols,rows]])

#원근 변환 행렬 계산
mtrx = cv2.getPerspectiveTransform(pts1, pts2)

#원근 변환 적용
dst_img = img.copy()
dst = cv2.warpPerspective(dst_img, mtrx, (cols,rows))

cv2.imshow('dst_IMG', dst)
cv2.waitKey()
cv2.destroyAllWindows()

#스캔 이미지처럼 만들어주기
scan_img = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
scan_img = cv2.adaptiveThreshold(scan_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21,10)

cv2.imshow('SCAN_IMG', scan_img)
cv2.waitKey()
cv2.destroyAllWindows()

[[[101  55]]

 [[ 23 706]]

 [[568 695]]

 [[497 103]]]
